In [1]:
from precomputed_python import AnnotationReader
import numpy as np

In [2]:
pre_ann=AnnotationReader("gs://neuroglancer-20191211_fafbv14_buhmann2019_li20190805")

I0328 10:15:41.159820 4528797 google_auth_provider.cc:149] Using credentials at /Users/forrestc/.config/gcloud/application_default_credentials.json
I0328 10:15:41.162372 4528797 google_auth_provider.cc:156] Using OAuth2 AuthProvider


In [3]:
annbytes = pre_ann.get_by_id(141187882)

In [4]:
np.frombuffer(annbytes)

array([2.51024744e+039, 7.77068963e+042, 2.34901230e+040, 6.55433771e+020,
       6.97560821e-316, 1.06356851e-104, 0.00000000e+000])

In [3]:
info=pre_ann.get_info()

In [3]:
import tensorstore

In [ ]:
import neuroglancer

'TensorStore is a library for reading and writing multi-dimensional arrays.'